In [ ]:
from IPython.display import display, HTML
display(HTML('''
<style>
.jp-Cell-outputWrapper .jp-Placeholder {
    display: none;
}
</style>
'''))

<center>
    <img src="https://upload.wikimedia.org/wikipedia/commons/a/a8/%D0%9B%D0%9E%D0%93%D0%9E_%D0%A8%D0%90%D0%94.png" width=500px/>
    <font>Python 2023</font><br/>
    <br/>
    <br/>
    <b style="font-size: 2em">Типы - Часть 2</b><br/>
    <br/>
    <font>Никита Бондарцев</font><br/>
</center>

### Что мы сегодня разберем?

### Специальный тип Any

In [ ]:
import typing as tp
print(tp.Any.__doc__.split("\n")[0])

### Почему тип Any специальный?

In [ ]:
import typing as tp

# Тип-класс
class A:
    pass

# Тип-класс
B = int

# Тип-объект
C = tp.Any

type(A), type(B), type(C)

In [ ]:
isinstance(1, tp.Any)

### Специальный тип Union

In [ ]:
import typing as tp
# нынче, с питона 3.10, можно писать просто через вертикальную черту A | B
print(tp.Union.__doc__.split("\n")[0])

Определение подтипа:
* ∀ A: A -> A
* A -> B  =>  A.values ⊇ B.values
* A -> B  =>  A.functions ⊆ B.functions

float | str => объединение всех значений, пересечение всех методов, поэтому

#### Выполняются ли такие свойства?

float | str -> float \
float | str -> str

In [ ]:
%%typecheck

class A:
    def am(self) -> None:
        pass
    
    def run(self) -> None:
        pass

class B:
    def run(self) -> None:
        pass


a: A | B = A()
reveal_type(a)
a.am()
a.run()

In [ ]:
float | str == str | float, tp.Union[str, float] == str | float

In [ ]:
float == tp.Union[float]

### Union → Примеры

In [ ]:
%%typecheck
# Использование надтипов в Union

def f(a: float | str):
    pass

f(2)
f(2.0)
f("hello")
f({})

In [ ]:
%%typecheck
# Использование подтипов в Union

def f(a: int | str):
    pass

f(2)
f(2.0)
f("hello")
f({})

In [ ]:
%%typecheck
# Вывод типов с учетом isinstance

def f(a: int | float | str):
    reveal_type(a)
    if isinstance(a, int):
        reveal_type(a)
    else:
        reveal_type(a)
        if isinstance(a, str):
            reveal_type(a)
        else:
            reveal_type(a)


In [ ]:
%%typecheck
# Вывод типов с учетом isinstance
# --warn-unreachable

def f(a: int | float | str):
    reveal_type(a)
    if isinstance(a, float):
        reveal_type(a)
    else:
        reveal_type(a)
        if isinstance(a, str):
            reveal_type(a)
        else:
            reveal_type(a)

### Специальный тип Optional

In [ ]:
import typing as tp
print(tp.Optional.__doc__.split("\n")[0])

In [ ]:
import typing as tp
from types import NoneType  # python 3.10+

tp.Optional[float] == float | None, tp.Optional[float] == float | NoneType, type(None), type(None) is NoneType

### Специальный тип Optional, примеры

In [19]:
%%typecheck

def f(a: float | None):
    pass

f(1)
f(1.0)
f("1")
f(None)

<string>:8: error: Argument 1 to "f" has incompatible type "str"; expected "float | None"  [arg-type]
Found 1 error in 1 file (checked 1 source file)


In [20]:
%%typecheck
# reveal в Optional делается аналогично Union

def f(a: float | None):
    reveal_type(a)
    if a is None:
        reveal_type(a)
    else:
        reveal_type(a)

<string>:5: note: Revealed type is "Union[builtins.float, None]"
<string>:7: note: Revealed type is "None"
<string>:9: note: Revealed type is "builtins.float"
Success: no issues found in 1 source file


### Что такое генерики?

Генерик - параметризированный тип, [ссылка на доку](https://mypy.readthedocs.io/en/stable/generics.html#defining-generic-classes)

Например, \
List[int] - тип list с параметром int \
Dict[str, int] - dict c параметром str у ключа и int у значения \
Tuple[int, ...] - tuple с параметрами int

Note: с питона 3.9 можно использовать стандартные контейнеры в качестве генериков, например, list[int], dict[str, int], tuple[int, float, str]

### Ковариантность/контрвариантность

### Примеры вариантности

### typing.Tuple (tuple)

In [21]:
import typing as tp
print(tp.Tuple.__doc__.split("\n")[0])

Deprecated alias to builtins.tuple.


In [23]:
%%typecheck
import typing as tp

a: tp.Tuple[int, str] = (1, "hello")
b: tuple[int, int, int] = (1, 2, 3)
c: tuple[int, ...] = (1, 2, 3, 4, 5)
d: tuple[int, float, str] = (1, 2.)

<string>:7: error: Incompatible types in assignment (expression has type "tuple[int, float]", variable has type "tuple[int, float, str]")  [assignment]
Found 1 error in 1 file (checked 1 source file)


### Tuple, пример

In [24]:
%%typecheck
# Heterogeneus tuple 

def f(a: tuple[int, float]):
    a[0] << 10
    a[1] << 10
    a[2] + 1

f((1, 1.4))
f((1, 1.4, 1))
f((1, 1))
f((1.4, 1))
f((1, "1"))

<string>:6: error: Unsupported operand types for << ("float" and "int")  [operator]
<string>:7: error: Tuple index out of range  [misc]
<string>:10: error: Argument 1 to "f" has incompatible type "tuple[int, float, int]"; expected "tuple[int, float]"  [arg-type]
<string>:12: error: Argument 1 to "f" has incompatible type "tuple[float, int]"; expected "tuple[int, float]"  [arg-type]
<string>:13: error: Argument 1 to "f" has incompatible type "tuple[int, str]"; expected "tuple[int, float]"  [arg-type]
Found 5 errors in 1 file (checked 1 source file)


In [25]:
%%typecheck
# Homogeneus tuple 

def f(a: tuple[float, ...]):
    a[0] + 1
    a[1] << 10
    a[100500] = 1.54

f((1, 2, 3, 4, 5))
f((1.1, 2.1, 3.1, 4.1, 5.1))
f(("3", "2", "1"))
f(tuple())

<string>:6: error: Unsupported operand types for << ("float" and "int")  [operator]
<string>:7: error: Unsupported target for indexed assignment ("tuple[float, ...]")  [index]
<string>:11: error: Argument 1 to "f" has incompatible type "tuple[str, str, str]"; expected "tuple[float, ...]"  [arg-type]
Found 3 errors in 1 file (checked 1 source file)


### Какая вариантность у Tuple?

### typing.List (list)

In [29]:
import typing as tp
print("\n".join(tp.List.__doc__.split("\n")[0:3]))

A generic version of list.


### List, примеры

In [30]:
%%typecheck

a = [1, "hello", 5.1]
reveal_type(a)

a.append(1)
a.append(1.0)
a.append("hi")
a.append([])

<string>:4: note: Revealed type is "builtins.list[builtins.object]"
Success: no issues found in 1 source file


In [31]:
%%typecheck

a = []
reveal_type(a)

<string>:3: error: Need type annotation for "a" (hint: "a: List[<type>] = ...")  [var-annotated]
<string>:4: note: Revealed type is "builtins.list[Any]"
Found 1 error in 1 file (checked 1 source file)


In [32]:
%%typecheck

a: list[float] = []

a.append(1)
reveal_type(a)

<string>:6: note: Revealed type is "builtins.list[builtins.float]"
Success: no issues found in 1 source file


In [33]:
%%typecheck

a: list[int] = []

a.append(1.1)
reveal_type(a)

<string>:5: error: Argument 1 to "append" of "list" has incompatible type "float"; expected "int"  [arg-type]
<string>:6: note: Revealed type is "builtins.list[builtins.int]"
Found 1 error in 1 file (checked 1 source file)


In [34]:
%%typecheck
# Повышение типа параметра

def foo(a: list[int]) -> None:
    pass

my_list = [1.1, 3.1, 5.1]
reveal_type(my_list)

foo(my_list)

<string>:8: note: Revealed type is "builtins.list[builtins.float]"
<string>:10: error: Argument 1 to "foo" has incompatible type "list[float]"; expected "list[int]"  [arg-type]
Found 1 error in 1 file (checked 1 source file)


In [35]:
%%typecheck
# Понижение типа параметра

def foo(a: list[float]) -> None:
    pass

my_list = [1, 2, 3]
reveal_type(my_list)

foo(my_list)

<string>:8: note: Revealed type is "builtins.list[builtins.int]"
<string>:10: error: Argument 1 to "foo" has incompatible type "list[int]"; expected "list[float]"  [arg-type]
<string>:10: note: "List" is invariant -- see https://mypy.readthedocs.io/en/stable/common_issues.html#variance
<string>:10: note: Consider using "Sequence" instead, which is covariant
Found 1 error in 1 file (checked 1 source file)


### Какая вариантность у List?

### typing.Sequence/typing.Mapping

### Sequence, пример

In [36]:
%%typecheck

def foo(a: list[float]) -> float:
    return a[0]

my_list = [1, 3, 5]

foo(my_list)

<string>:8: error: Argument 1 to "foo" has incompatible type "list[int]"; expected "list[float]"  [arg-type]
<string>:8: note: "List" is invariant -- see https://mypy.readthedocs.io/en/stable/common_issues.html#variance
<string>:8: note: Consider using "Sequence" instead, which is covariant
Found 1 error in 1 file (checked 1 source file)


In [37]:
%%typecheck

import collections.abc as abc

def foo(a: abc.Sequence[float]) -> float:
    return a[0]

my_list = [1, 3, 5]

foo(my_list)

Success: no issues found in 1 source file


### Mapping, пример

In [38]:
%%typecheck

def foo(a: dict[str, float]) -> float:
    return a["key"]

my_dict = {"hey": 1}

foo(my_dict)

<string>:8: error: Argument 1 to "foo" has incompatible type "dict[str, int]"; expected "dict[str, float]"  [arg-type]
<string>:8: note: "Dict" is invariant -- see https://mypy.readthedocs.io/en/stable/common_issues.html#variance
<string>:8: note: Consider using "Mapping" instead, which is covariant in the value type
Found 1 error in 1 file (checked 1 source file)


In [39]:
%%typecheck

import collections.abc as abc

def foo(a: abc.Mapping[str, float]) -> float:
    return a["key"]

my_dict = {"hey": 1}

foo(my_dict)

Success: no issues found in 1 source file


### Иерархия типов генериков
<img src="images/hierarchy2.jpg">

### Примеры на иерархию Sequence/Mapping 

In [40]:
%%typecheck

import collections.abc as abc


def foo1(a: abc.Sequence[float]) -> None:
    pass

def foo2(a: abc.MutableSequence[float]) -> None:
    pass

def foo3(a: list[float]) -> None:
    pass

def foo4(a: list) -> None:
    pass


a = [1]
foo1(a)
foo2(a)
foo3(a)
foo4(a)


<string>:21: error: Argument 1 to "foo2" has incompatible type "list[int]"; expected "MutableSequence[float]"  [arg-type]
<string>:22: error: Argument 1 to "foo3" has incompatible type "list[int]"; expected "list[float]"  [arg-type]
<string>:22: note: "List" is invariant -- see https://mypy.readthedocs.io/en/stable/common_issues.html#variance
<string>:22: note: Consider using "Sequence" instead, which is covariant
Found 2 errors in 1 file (checked 1 source file)


In [41]:
%%typecheck

import collections.abc as abc


def foo1(a: abc.Mapping[str, float]) -> None:
    pass

def foo2(a: abc.MutableMapping[str, float]) -> None:
    pass

def foo3(a: dict[str, float]) -> None:
    pass

def foo4(a: dict) -> None:
    pass


a = {'a': 1}
foo1(a)
foo2(a)
foo3(a)
foo4(a)

<string>:21: error: Argument 1 to "foo2" has incompatible type "dict[str, int]"; expected "MutableMapping[str, float]"  [arg-type]
<string>:22: error: Argument 1 to "foo3" has incompatible type "dict[str, int]"; expected "dict[str, float]"  [arg-type]
<string>:22: note: "Dict" is invariant -- see https://mypy.readthedocs.io/en/stable/common_issues.html#variance
<string>:22: note: Consider using "Mapping" instead, which is covariant in the value type
Found 2 errors in 1 file (checked 1 source file)


### Generic functions, TypeVar

In [42]:
import typing
tp.TypeVar.__doc__.split("\n")[0]

'Type variable.'

#### TypeVar, мотивация

In [43]:
%%typecheck

def foo(a: int, b: int) -> int:
    reveal_type(a)
    return a + b

a = foo(1, 2)
reveal_type(a)

<string>:4: note: Revealed type is "builtins.int"
<string>:8: note: Revealed type is "builtins.int"
Success: no issues found in 1 source file


In [44]:
%%typecheck

def foo(a: int, b: int) -> int:
    return a + b

def foo2(a: str, b: str) -> str:
    return a + b

a = foo(1, 2)
reveal_type(a)

b = foo2("1", "2")
reveal_type(b)

c = foo("1", 2)
reveal_type(c)

<string>:10: note: Revealed type is "builtins.int"
<string>:13: note: Revealed type is "builtins.str"
<string>:15: error: Argument 1 to "foo" has incompatible type "str"; expected "int"  [arg-type]
<string>:16: note: Revealed type is "builtins.int"
Found 1 error in 1 file (checked 1 source file)


In [45]:
%%typecheck

def foo(a: int | str, b: int | str) -> int | str:
    reveal_type(a)
    return a + b

a = foo(1, 2)
reveal_type(a)

b = foo("1", "2")
reveal_type(b)

c = foo("1", 2)
reveal_type(c)


<string>:4: note: Revealed type is "Union[builtins.int, builtins.str]"
<string>:5: error: Unsupported operand types for + ("int" and "str")  [operator]
<string>:5: error: Unsupported operand types for + ("str" and "int")  [operator]
<string>:5: note: Both left and right operands are unions
<string>:8: note: Revealed type is "Union[builtins.int, builtins.str]"
<string>:11: note: Revealed type is "Union[builtins.int, builtins.str]"
<string>:14: note: Revealed type is "Union[builtins.int, builtins.str]"
Found 2 errors in 1 file (checked 1 source file)


### TypeVar, примеры

In [46]:
%%typecheck

import typing as tp

T = tp.TypeVar('T', int, str)

def foo(a: T, b: T) -> T:
    reveal_type(a)
    return a + b

a = foo(1, 2)
reveal_type(a)

b = foo("1", "2")
reveal_type(b)

c = foo("1", 2)
reveal_type(c)

<string>:8: note: Revealed type is "builtins.int"
<string>:8: note: Revealed type is "builtins.str"
<string>:12: note: Revealed type is "builtins.int"
<string>:15: note: Revealed type is "builtins.str"
<string>:17: error: Value of type variable "T" of "foo" cannot be "object"  [type-var]
<string>:18: note: Revealed type is "builtins.object"
Found 1 error in 1 file (checked 1 source file)


In [47]:
%%typecheck

import typing as tp

T = tp.TypeVar('T')

def foo(a: list[T], n: int) -> T:
    return a[n]

a = foo([1, 2], 0)
reveal_type(a)

<string>:11: note: Revealed type is "builtins.int"
Success: no issues found in 1 source file


### Overload - не то, что вы подумали
[дока](https://peps.python.org/pep-0484/#function-method-overloading)

In [34]:
%%typecheck

lst = [1, 2, 3]
reveal_type(lst[0])
reveal_type(lst[0:1])

<string>:4: note: Revealed type is "builtins.int"
<string>:5: note: Revealed type is "builtins.list[builtins.int]"
Success: no issues found in 1 source file


In [37]:
%%typecheck

import typing as tp

class MyIntList:
    def __init__(self, lst: list[int]) -> None:
        self.lst = lst

    @tp.overload
    def __getitem__(self, idx: slice) -> list[int]:
        pass

    @tp.overload
    def __getitem__(self, idx: int) -> int:
        pass
    
    def __getitem__(self, idx):
        return self.lst[idx]

my_lst = MyIntList([1, 2, 3])
reveal_type(my_lst[0])
reveal_type(my_lst[0:1])

<string>:21: note: Revealed type is "builtins.int"
<string>:22: note: Revealed type is "builtins.list[builtins.int]"
Success: no issues found in 1 source file


### Функциональный тип Callable

In [60]:
%%typecheck

import collections.abc as abc

def g(a: int, b: float) -> float:
    return 1.1

a: abc.Callable[[int, float], float] = g

Success: no issues found in 1 source file


### Callable, аннотация типов функции совпадает с тайпингом

In [61]:
%%typecheck
# Привычная функция

import collections.abc as abc

def f(a: abc.Callable[[int, float], float]) -> None:
    pass

def g(a: int, b: float) -> float:
    return 1.1

f(g)

Success: no issues found in 1 source file


In [62]:
%%typecheck
# Функция без аргументов

import collections.abc as abc

def f(a: abc.Callable[[], float]):
    pass

def g() -> float:
    return 1.1

f(g)

Success: no issues found in 1 source file


### Callable, часть типов отсутствует

In [63]:
%%typecheck
# Любые аргументы

import collections.abc as abc

def f(a: abc.Callable[..., float]):
    pass

def g(a: int, b: str, c: int) -> float:
    return 1.1

f(g)

Success: no issues found in 1 source file


In [64]:
%%typecheck
# И лямбда тоже

import typing as tp

def f(a: tp.Callable[..., float]):
    pass

f(lambda a, b, c: 1.1)

Success: no issues found in 1 source file


### Callable, передаем функции с другими типами аргументов

In [65]:
%%typecheck
# Повышаем тип аргумента

import collections.abc as abc

def f(a: abc.Callable[[int], float]):
    pass

def g(a: float) -> float:
    return a

f(g)

Success: no issues found in 1 source file


In [66]:
%%typecheck
# Понижаем тип аргумента

import collections.abc as abc

def f(a: abc.Callable[[float], float]):
    pass

def g(a: int) -> float:
    return a

f(g)

<string>:12: error: Argument 1 to "f" has incompatible type "Callable[[int], float]"; expected "Callable[[float], float]"  [arg-type]
Found 1 error in 1 file (checked 1 source file)


### Callable, передаем функции с другим типом возвращаемого значения

In [67]:
%%typecheck
# Повышаем тип возвращаемого значения

import collections.abc as abc

def f(a: abc.Callable[[int], int]):
    pass

def g(a: int) -> float:
    return a

f(g)

<string>:12: error: Argument 1 to "f" has incompatible type "Callable[[int], float]"; expected "Callable[[int], int]"  [arg-type]
Found 1 error in 1 file (checked 1 source file)


In [68]:
%%typecheck
# Понижаем тип возвращаемого значения

import collections.abc as abc

def f(a: abc.Callable[[int], float]):
    pass

def g(a: int) -> int:
    return a

f(g)

Success: no issues found in 1 source file


### Какая вариантность Callable по аргументам? По возвращаемому значению?

### Создание собственных Generic классов

In [69]:
import typing as tp
print(tp.Generic.__doc__.split('\n')[0])

Abstract base class for generic types.


### Generic, не так прост как кажется

In [70]:
%%typecheck
import typing as tp

# Используем тайпвар, как будто пишем дженерик функцию - не работает

T = tp.TypeVar("T", str, int)

class A:
    def __init__(self, a: T) -> None:
        self._a = a
        reveal_type(self._a)
        
    def am(self) -> T:
        reveal_type(self._a)
        return self._a + self._a

    
a = A(1)
reveal_type(a)
b = a.am()
reveal_type(b)

<string>:8: error: Need type annotation for "_a"  [var-annotated]
<string>:9: note: Revealed type is "builtins.str"
<string>:9: note: Revealed type is "Any"
<string>:12: note: Revealed type is "Any"
<string>:17: note: Revealed type is "__main__.A"
<string>:19: note: Revealed type is "builtins.str"
Found 1 error in 1 file (checked 1 source file)


In [71]:
%%typecheck
import typing as tp

T = tp.TypeVar("T", int, str)

class A(tp.Generic[T]):    
    def __init__(self, a: T) -> None:
        self._a: T = a
        reveal_type(self._a)
        
    def am(self) -> T:
        reveal_type(self._a)
        return self._a


a = A(1)
reveal_type(a)
b = a.am()
reveal_type(b)

c = A("hello")
reveal_type(c)
d = c.am()
reveal_type(d)

<string>:9: note: Revealed type is "builtins.int"
<string>:9: note: Revealed type is "builtins.str"
<string>:12: note: Revealed type is "builtins.int"
<string>:12: note: Revealed type is "builtins.str"
<string>:17: note: Revealed type is "__main__.A[builtins.int]"
<string>:19: note: Revealed type is "builtins.int"
<string>:22: note: Revealed type is "__main__.A[builtins.str]"
<string>:24: note: Revealed type is "builtins.str"
Success: no issues found in 1 source file


In [72]:
%%typecheck

import typing as tp

T = tp.TypeVar("T")

class A(tp.Generic[T]):
    def __init__(self) -> None:
        self._a: list[T] = []
        
    def add(self, a: T) -> None:
        self._a.append(a)


a: A[int] = A()
a.add(1)
reveal_type(a)

b: A[float] = A()
b.add("sss")

<string>:17: note: Revealed type is "__main__.A[builtins.int]"
<string>:20: error: Argument 1 to "add" of "A" has incompatible type "str"; expected "float"  [arg-type]
Found 1 error in 1 file (checked 1 source file)


###  Переменные классов, Type

In [90]:
# https://www.python.org/dev/peps/pep-0484/#the-type-of-class-objects

import typing as tp
print(tp.Type.__doc__.split("\n")[0])

Deprecated alias to builtins.type.


In [91]:
%%typecheck

import typing as tp

a: tp.Type[int] = int

class B:
    pass

b: tp.Type[B] = B

Success: no issues found in 1 source file


###  Type, пример

In [92]:
%%typecheck
import typing as tp

class A:
    pass

class B(A):
    pass

def foo(a: tp.Type[A]) -> None:
    pass


foo(A())
foo(A)
foo(B())
foo(B)

<string>:14: error: Argument 1 to "foo" has incompatible type "A"; expected "type[A]"  [arg-type]
<string>:16: error: Argument 1 to "foo" has incompatible type "B"; expected "type[A]"  [arg-type]
Found 2 errors in 1 file (checked 1 source file)


In [93]:
%%typecheck
import typing as tp

class A:
    def __init__(self, a: int) -> None:
        self.a = a

    @classmethod
    def build(cls: tp.Type[A]) -> A:
        return cls(1)

class B(A):
    pass
    

reveal_type(A.build())
reveal_type(B.build())

<string>:16: note: Revealed type is "__main__.A"
<string>:17: note: Revealed type is "__main__.A"
Success: no issues found in 1 source file


### Ограничение TypeVar без необходимости перечислять все варианты

In [95]:
%%typecheck
import typing as tp

T = tp.TypeVar('T')

class A:
    def __init__(self, a: int) -> None:
        self.a = a

    @classmethod
    def build(cls: tp.Type[T]) -> T:
        return cls(1)

class B(A):
    pass
    

reveal_type(A.build())
reveal_type(B.build())

<string>:12: error: Too many arguments for "object"  [call-arg]
<string>:18: note: Revealed type is "__main__.A"
<string>:19: note: Revealed type is "__main__.B"
Found 1 error in 1 file (checked 1 source file)


###  Какая вариантность у Type?

### Типы → Nominal subtyping

### Типы → Structural subtyping, немного черной магии

В целом, в питончике творится черная магия с числовыми типами. Есть специальная [иерархия абстрактных типов](https://peps.python.org/pep-3141/) для чисел. А обычные числовые типы являются не прямыми наследниками друг драга, а скорее [структурными](https://mypy.readthedocs.io/en/latest/duck_type_compatibility.html).

In [45]:
from numbers import Real, Integral

isinstance(1, Real), isinstance(1, Integral), isinstance(1.1, Integral)

(True, True, False)

In [46]:
issubclass(float, Real), issubclass(int, float), issubclass(float, Integral)

(True, False, False)

Если кому-то интересно что за чертовщина происходит в ячейке ниже, велкам [в доку](https://peps.python.org/pep-0544/#existing-approaches-to-structural-subtyping). А вообще, с хеллоуином вас, товарищи!

In [51]:
%%typecheck

from abc import ABC, abstractmethod

class MyAbstract(ABC):
    @abstractmethod
    def haha(self) -> str:
        pass

class A:
    pass

MyAbstract.register(A)

isinstance(A(), MyAbstract)

Success: no issues found in 1 source file


### Structural subtyping, примеры

In [21]:
%%typecheck
import collections.abc as abc
import typing as tp
# проставьте тайпинг, чтобы не упало

def validate_size(a, n) -> None:
    if (a_len := len(a)) > n:
        raise ValueError(
            f"Structure length {a_len} is greater then expected {n}"
        )

        
validate_size([10, 11], 1)
validate_size(1010, 1)

Success: no issues found in 1 source file


In [22]:
%%typecheck
import collections.abc as abc
import typing as tp
# проставьте тайпинг, чтобы не упало

def validate_size(a, n) -> None:
    if (a_len := len(a)) > n:
        raise ValueError(
            f"Structure length {a_len} is greater then expected {n}"
        )

        
validate_size([10, 11], 1)
validate_size((1, 3, 10), 1)
validate_size(1010, 1)


Success: no issues found in 1 source file


In [23]:
%%typecheck
import collections.abc as abc
import typing as tp
# проставьте тайпинг, чтобы не упало

def validate_size(a, n) -> None:
    if (a_len := len(a)) > n:
        raise ValueError(
            f"Structure length {a_len} is greater then expected {n}"
        )
        
validate_size([10, 11], 1)
validate_size((1, 3, 10), 1)
validate_size({1, 3, 10}, 1)
validate_size(1010, 1)

Success: no issues found in 1 source file


In [24]:
%%typecheck
import collections.abc as abc
import typing as tp
# проставьте тайпинг, чтобы не упало

class A:
    def __len__(self):
        return 1

def validate_size(a, n) -> None:
    if (a_len := len(a)) > n:
        raise ValueError(
            f"Structure length {a_len} is greater then expected {n}"
        )
        
validate_size([10, 11], 1)
validate_size((1, 3, 10), 1)
validate_size({1, 3, 10}, 1)
validate_size(A(), 1)
validate_size(1010, 1)



Success: no issues found in 1 source file


### Собственный Structural Subtping - Protocol

In [25]:
%%typecheck

class A:
    def close(self):
        print("Close enough")

class B:
    pass


c = open("my_file")


def foo(a) -> None:
    reveal_type(a)
    a.close()


foo(A())
foo(B())
foo(c)

<string>:15: note: Revealed type is "Any"
Success: no issues found in 1 source file


In [26]:
%%typecheck

import typing as tp


class Closeable(tp.Protocol):
    def close(self) -> None:
        pass
    

class A:
    def close(self):
        print("Close enough")

class B:
    pass


c = open("my_file")


def foo(a: Closeable) -> None:
    reveal_type(a)
    a.close()


foo(A())
foo(B())
foo(c)

<string>:23: note: Revealed type is "__main__.Closeable"
<string>:28: error: Argument 1 to "foo" has incompatible type "B"; expected "Closeable"  [arg-type]
Found 1 error in 1 file (checked 1 source file)


### Protocol and isinstance, runtime_checkable

In [27]:
import typing as tp

# Note that instance checks are not 100% reliable statically, this is why this behavior is opt-in, see section on rejected ideas for examples.
# @see https://peps.python.org/pep-0544/#runtime-checkable-decorator-and-narrowing-types-by-isinstance

@tp.runtime_checkable
class Closeable(tp.Protocol):
    def close(self) -> None:
        pass


class A:
    def close(self):
        print("Close enough")

isinstance(A(), Closeable)

True

### Пример готового Generic + Protocol, просто чтобы вас запутать

In [29]:
%%typecheck

import typing as tp
import collections.abc as abc

def foo(s: abc.Iterable[int]) -> None:
    pass

class A:
    pass

T = tp.TypeVar("T")

class B(tp.Generic[T]):
    def __iter__(self) -> abc.Iterator[T]:
        return iter([])
    


foo(A())

b: B[int] = B()
foo(b)

c: B[str] = B()
foo(c)

foo([])

<string>:20: error: Argument 1 to "foo" has incompatible type "A"; expected "Iterable[int]"  [arg-type]
<string>:26: error: Argument 1 to "foo" has incompatible type "B[str]"; expected "Iterable[int]"  [arg-type]
Found 2 errors in 1 file (checked 1 source file)
